In [29]:
from pymongo import MongoClient
from bson.code import Code
import pandas as pd
import re

client = MongoClient("mongodb://bigdata-mongodb-04.virtual.uniandes.edu.co:8087/", retryWrites=False)
database = client["Grupo03"]
collection = database["COL_dataset"]


In [45]:
query = {}
projection = {}
projection["_id"] = 1.0
projection["reply_or_quote"] = 1.0
projection["user"] = 1.0
projection["tendencia"] = 1.0

data = []
cursor = collection.find(query, projection = projection)
try:
    for doc in cursor:
        try:
            data.append([doc['_id'], doc['user'], doc['reply_or_quote'], doc['tendencia']])
        except KeyError as e:
            data.append([doc['_id'], doc['user'], doc['reply_or_quote'], ''])
        
finally:
    client.close()

In [46]:
df = pd.DataFrame(data,columns=['_id', 'influencer', 'tweet', 'tendencia'])
df.head(150)
def quitar_cuentas(a):
    texto=" ".join(filter(lambda x:x[0]!='@', a.split()))
    return texto


In [47]:
from joblib import dump, load
from sklearn.feature_extraction.text import CountVectorizer
clf_col = load('/home/jcardenasc93/Documents/MISO/BigData/Repo/DatosEnormes/Script/twitter/cotweet/assets/pys/modelo_tendencia_col.joblib') 
loaded_vec_col = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("/home/jcardenasc93/Documents/MISO/BigData/Repo/DatosEnormes/Script/twitter/cotweet/assets/pys/vocabulario_tendencia_col.pkl", "rb")))


/home/jcardenasc93/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [48]:
tendencias =[["Apoyo",0],
             ["Contradicción",1],
             ["Matoneo",2]]

## Poner el label de la emoción seg+un el número
def label_tendencia(number):
    tendencia = ''
    for i in range(len(tendencias)):
        if tendencias[i][1] == number:
            tendencia = tendencias[i][0]
    return tendencia

In [49]:
df['prediccion'] = df['tweet'].apply(lambda x: label_tendencia(int(clf_col.predict(loaded_vec_col.transform([quitar_cuentas(x)]))[0])))

In [50]:
df.head()

,_id,influencer,tweet,tendencia,prediccion
0,5e95c25095e4b2cefb66fbf2,ClaudiaLopez,@ClaudiaLopez @infopresidencia @Bogota Para cu...,,Contradicción
1,5e95c25095e4b2cefb66fbf3,ClaudiaLopez,@ClaudiaLopez @infopresidencia @Bogota La verd...,,Contradicción
2,5e95c25095e4b2cefb66fbf4,ClaudiaLopez,"@ClaudiaLopez @infopresidencia @Bogota Pero, x...",,Contradicción
3,5e95c25095e4b2cefb66fbf5,ClaudiaLopez,@ClaudiaLopez @infopresidencia @Bogota Ingreso...,,Contradicción
4,5e95c25095e4b2cefb66fbf6,ClaudiaLopez,@ClaudiaLopez @infopresidencia @Bogota Estimad...,,Contradicción


In [51]:
df = df.groupby(['prediccion', 'influencer']).count().reset_index()

In [52]:
df.head(100)

,prediccion,influencer,_id,tweet,tendencia
0,Apoyo,AlvaroUribeVel,420,420,420
1,Apoyo,BluRadioCo,44,44,44
2,Apoyo,ClaudiaLopez,90,90,90
3,Apoyo,GustavoBolivar,23,23,23
4,Apoyo,IvanDuque,20,20,20
5,Apoyo,JorgeIvanOspina,66,66,66
6,Apoyo,MabelLaraNews,51,51,51
7,Apoyo,MinSaludCol,40,40,40
8,Apoyo,NoticiasCaracol,61,61,61
9,Apoyo,NoticiasRCN,50,50,50


In [70]:
import plotly.express as px
fig_1 = px.treemap(df, path=['prediccion', 'influencer'], values='tendencia', title='Tendencia general')
fig_2 = px.treemap(df, path=['influencer', 'prediccion'], values='tendencia', title='Tendencia por influencer')

In [71]:
fig_1.show()

In [72]:
fig_2.show()